In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Cargar el archivo CSV limpio
file_path = 'archivo/data_CLEAN.csv'  # Cambia esta ruta por la ruta correcta a tu archivo limpio
df_clean = pd.read_csv(file_path)

# Variables predictoras y objetivo
X = df_clean[['rooms', 'bathroom', 'lift', 'terrace', 'square_meters', 'neighborhood', 'real_state']]
y = df_clean['price']

# Convertir las variables categóricas en numéricas con One-Hot Encoding
categorical_features = ['neighborhood', 'real_state']

# Crear un preprocesador con OneHotEncoder para las variables categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Dividir el dataset en conjunto de entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo XGBoost en un pipeline con el preprocesador
xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(objective='reg:squarederror', random_state=42))
])

# Definir los hiperparámetros a ajustar en RandomizedSearchCV
param_distributions = {
    'model__n_estimators': [100, 200, 300, 500],
    'model__max_depth': [3, 5, 7, 10],
    'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'model__subsample': [0.6, 0.8, 1.0],
    'model__colsample_bytree': [0.6, 0.8, 1.0],
    'model__gamma': [0, 0.1, 0.2, 0.3]
}

# Configurar RandomizedSearchCV para buscar los mejores hiperparámetros
random_search_xgb = RandomizedSearchCV(
    xgb_model, 
    param_distributions=param_distributions, 
    n_iter=50,  # Número de iteraciones aleatorias
    cv=3,  # Validación cruzada de 3 pliegues
    scoring='neg_mean_squared_error',  # Métrica para optimizar
    random_state=42,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Entrenar RandomizedSearchCV
random_search_xgb.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados y entrenar el modelo final
best_xgb_model = random_search_xgb.best_estimator_
y_pred_xgb = best_xgb_model.predict(X_test)

# Calcular las medidas de precisión del modelo ajustado
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Mostrar las nuevas medidas de precisión
precision_measures_xgb = {
    'RMSE': rmse_xgb,
    'MAE': mae_xgb,
    'R2 Score': r2_xgb
}
print("Medidas de precisión del modelo XGBoost ajustado:", precision_measures_xgb)

# Visualizar la comparación entre los valores reales y los predichos
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_xgb, alpha=0.5, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2, linestyle='--')
plt.title('Comparación de Valores Reales vs. Predichos (XGBoost Ajustado)')
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.grid(True)
plt.show()

ModuleNotFoundError: No module named 'xgboost'